# Imports

In [1]:
import math
import numpy as np
from matplotlib import pyplot as plt
from numpy import random as rnd
import warnings,datetime,os,calendar,csv,time

import torch
from torch import nn
from torch import optim
from torch.nn import functional as F 

import pandas as pd
import seaborn as sns

import gym
from bs4 import BeautifulSoup

from sklearn import datasets
from sklearn.linear_model import LinearRegression

warnings.filterwarnings('ignore')
sns.set()

# Load and preprocess data

In [2]:
rootpath = '..\\..\\TF_data\\STOCKDATA\\Data\\Stocks\\'
available_indices = os.listdir(rootpath)

In [3]:
bf = pd.read_csv(rootpath+available_indices[0]).set_index(['Date'])

In [4]:
LOOKBACK = 8
LOOKFORWARD = 1

TARGET_FEATURE_ROW = 0

X = []
y = []

V = bf.values
for k in range(bf.shape[0]-LOOKBACK-LOOKFORWARD+1):
    X.append(V[k:k+LOOKBACK,:])
    y.append(V[k+LOOKBACK+(LOOKFORWARD-1),TARGET_FEATURE_ROW])

X = np.stack(X,axis=0)
y = np.stack(y,axis=0)

In [5]:
test_ratio = 0.8

X0 = X[:round(X.shape[0]*test_ratio),:,:]
X1 = X[round(X.shape[0]*test_ratio):,:,:]

Y0 = y[:round(X.shape[0]*test_ratio)]
Y1 = y[round(X.shape[0]*test_ratio):]

# Build baseline predictors

In [6]:
n_timesteps = X.shape[1]
n_features = X.shape[2]

In [7]:
rf = pd.DataFrame()
for k in range(X1[:,:,0].shape[1]):
    rf['T-'+str(X1[:,:,0].shape[1]-k)] = X1[:,k,0]

rf['Real'] = Y1

rf['Pred DR0'] = rf['T-1']
rf['SQE DR0'] = rf[['Pred DR0','Real']].apply(lambda x: (x[0]-x[1])**2,axis=1)

In [8]:
lr_clf = LinearRegression()
lr_clf.fit(X0[:,:,0],Y0)

rf['Pred LR'] = lr_clf.predict(X1[:,:,0])
rf['SQE LR'] = rf[['Pred LR','Real']].apply(lambda x: (x[0]-x[1])**2,axis=1)

In [9]:
rf[['SQE DR0','SQE LR']].mean()

SQE DR0    0.341097
SQE LR     0.368741
dtype: float64

# Simple Neural Network Forecast

In [10]:
torch.manual_seed(1)

In [11]:
class Net(nn.Module):
    # Build neural network skeleton
    def __init__(self):
        super(Net,self).__init__()
        self.l0 = nn.Linear(n_timesteps,15)
        self.l1 = nn.Linear(15,10)
        self.l2 = nn.Linear(10,1)
    # Define data flow through the network
    def forward(self,x):
        x = F.relu(self.l0(x))
        x = F.relu(self.l1(x))
        x = F.linear(self.l2(x),weight=torch.ones(1))
        return x

In [12]:
net = Net()
optimizer = optim.Adam(net.parameters(),lr=0.01)
criterion = nn.MSELoss()

In [139]:
optimizer.zero_grad()
yh = net(torch.tensor(X0[:5,:,0]).float())
loss = criterion(yh,torch.tensor(Y0[:5]).float())

loss.backward()
optimizer.step()

In [140]:
loss

tensor(4.7868, grad_fn=<MseLossBackward>)

In [141]:
yh

tensor([32.1423, 31.6584, 31.6399, 32.1644, 32.1806], grad_fn=<MvBackward>)

In [142]:
Y0[:5]

array([28.48 , 29.532, 30.336, 30.547, 30.883])